# Shape-Only
Here we only extract the shape, by colorizing the lesion segmentation images by the dataset image mean, to reduce the domain shift.

In [1]:
import os
import cv2
import numpy as np

datasets = [
    {
        'dataset_name': 'Derm7pt',
        'segpath': 'Derm7pt/segmentations',
        'dstpath': 'Derm7pt/augmentations',
        'mean_color': [0.7579, 0.6583, 0.5911],
        'replace_strings': ['segmentations', 'augmentations/segmentation'],
    },
    {
        'dataset_name': 'ISIC',
        'segpath': 'ISIC/segmentations',
        'dstpath': 'ISIC/augmentations',
        'mean_color': [0.6809, 0.5183, 0.5192],
        'replace_strings': ['segmentations', 'augmentations/segmentation'],
    }
]

for setting in datasets:
    gen = np.random.default_rng(seed=42)

    dataset_name = setting['dataset_name']
    segpath = setting['segpath']
    dstpath = setting['dstpath']
    mean_color = np.array(setting['mean_color'])
    replace_strings = setting['replace_strings']

    lst_filenames = []
    for root, dirs, files in os.walk(segpath):
        files = [ file for file in files if not file.endswith( ('.txt','.tar') ) ]
        for filename in files:
            
            abs_filepath = os.path.join(root, filename)
            
            img = cv2.imread(abs_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ret, img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

            color1 = mean_color - gen.uniform(0.01, 0.2, 1)
            color2 = 2*mean_color - color1

            mask = img == 0
            mask = mask[:,:,0]

            img[mask] = color2*255
            img[~mask] = color1*255

            img = cv2.blur(img, (5, 5))

            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            dst_filepath = abs_filepath.replace(replace_strings[0], replace_strings[1])

            # Save processed image
            basedir = '/'.join(dst_filepath.split('/')[:-1])
                        
            if not os.path.exists(basedir):
                os.makedirs(basedir)
                
            cv2.imwrite(dst_filepath, img)

